In [ ]:
Install And Import Dependencies

In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [1]:
from transformers import AutoTokenizer , AutoModelForSequenceClassification
import torch 
import requests 
from bs4 import BeautifulSoup
import re

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

C:\Users\bchai\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bchai\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [10]:
tokens =  tokenizer.encode('I love this , it is absolutlesly amazing' , return_tensors='pt')

In [11]:
tokens

tensor([[  101,   151, 11157, 10372,   117, 10197, 10127, 79244, 11300, 10563,
         39854,   102]])

In [12]:
tokenizer.decode(tokens[0])

'[CLS] i love this, it is absolutlesly amazing [SEP]'

In [13]:
result = model(tokens)

In [14]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.2922, -2.6076, -1.1806,  1.2227,  3.9803]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [15]:
int(torch.argmax(result.logits))+1

5

In [16]:
r = requests.get('https://www.yelp.com/biz/azteca-d-oro-orlando-2')
soup = BeautifulSoup(r.text , 'html.parser')
results = soup.find_all('p', class_='css-qgunke')
reviews = [result.text for result in results]

In [17]:
reviews[0]

'Lani B. said"I\'ve heard about this place from other Yelpers and saw pictures posted recently, so I decided to stop in when in the area. I must have had a blank look on my face as the young lady behind the counter immediately asked if this was my…"read more'

In [18]:
import numpy as np
import pandas as pd

In [21]:
df = pd.DataFrame(np.array(reviews),columns=['review'])

In [22]:
df.tail()

,review
10,When in Orlando and crave Mexican food this is...
11,Very loud. Couldn't hear the person across the...
12,Stopped here after a long day with my fiance. ...
13,"Lisa G. said""We have stayed here around 6 time..."
14,"Brittany F. said""Five stars for a great locati..."


In [23]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [24]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [25]:
df

,review,sentiment
0,"Lani B. said""I've heard about this place from ...",1
1,"Javon M. said""First of all you ever have a pla...",5
2,"I just recently got to eat at this location, a...",5
3,"Food was good, got the fajita quesadilla and l...",4
4,I came here for Happy Hour and sat at the bar...,2
5,Prices went up and service depends who serve y...,2
6,Since they did a remodel the bar and restauran...,5
7,From the atmosphere to food and drinks. The dr...,5
8,I've lived in Orlando for years and I have nev...,5
9,KUDOS and Thank you so much to Daniella for pr...,5
